In [1]:
import pandas as pd
import folium
import json
import branca
import numpy as np
import requests
from folium import plugins
import missingno as msno
from branca.colormap import linear
import matplotlib as mpl
import matplotlib.pyplot as plt

In [26]:
store19 = pd.read_csv('store2019.csv',encoding='utf-8')
store20 = pd.read_csv('store2020.csv',encoding='utf-8')
store19.head()

,adress,Latitude,Longitude,구,year,Q,store_type_code,store_type,adress_code,service_type_code,...,time6_count,male_count,female_count,age10_count,age20_count,age30_count,age40_count,age50_count,age60_count,store_count
0,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS300015,...,0.0,47.0,70.0,0.0,0.0,0.0,0.0,117.0,0.0,3.0
1,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100002,...,32.0,863.0,1254.0,174.0,815.0,471.0,424.0,179.0,53.0,1.0
2,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100005,...,1369.0,9568.0,12229.0,408.0,3952.0,7011.0,5095.0,3328.0,2002.0,5.0
3,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100004,...,100.0,1605.0,1839.0,31.0,1183.0,769.0,585.0,653.0,223.0,5.0
4,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100010,...,338.0,13176.0,27821.0,677.0,13792.0,13387.0,6605.0,4474.0,2061.0,21.0


In [160]:
#지도데이터 불러오기
url = 'https://raw.githubusercontent.com/suanlab/dataset/master'
seoul_geo = f'{url}/seoul_municipalities_geo_simple.json'
seoul_population = f'{url}/seoul_population.csv'

#알고싶은데이터는19년과 20년의 시간대별 매출양 차이
_19minor20=store19.iloc[:,68:74].groupby(store19['구']).sum() -\
store20.iloc[:,68:74].groupby(store19['구']).sum()

#인덱싱 재처리
_19minor20=_19minor20.reset_index()
_19minor20.mean()

time1_count    1313229.20
time2_count     -85091.24
time3_count      24198.68
time4_count     -78042.48
time5_count    2852563.96
time6_count    3027490.72
dtype: float64

In [161]:
# 데이터의 값을 측정해서 색 비율을 만들어준다.
# time1_count 값
colormap = linear.Blues_09.scale(
    _19minor20.time6_count.min(),
    _19minor20.time6_count.max()
)
colormap

In [158]:
#구별로 색을 입히자
_19minor20_dict = _19minor20.set_index('구')['time6_count'] #인덱스 
color_dict = {str(key): colormap(_19minor20_dict[key]) for key in _19minor20_dict.keys()}
color_dict

{'강남구': '#083471ff',
 '강동구': '#f7fbffff',
 '강북구': '#f7fbffff',
 '강서구': '#f7fbffff',
 '관악구': '#b3d3e9ff',
 '광진구': '#f7fbffff',
 '구로구': '#c6dbefff',
 '금천구': '#f7fbffff',
 '노원구': '#f7fbffff',
 '도봉구': '#f7fbffff',
 '동대문구': '#f7fbffff',
 '동작구': '#f7fbffff',
 '마포구': '#08306bff',
 '서대문구': '#e5f0faff',
 '서초구': '#60a7d2ff',
 '성동구': '#f7fbffff',
 '성북구': '#f7fbffff',
 '송파구': '#d5e5f4ff',
 '양천구': '#f7fbffff',
 '영등포구': '#f7fbffff',
 '용산구': '#dceaf7ff',
 '은평구': '#f7fbffff',
 '종로구': '#aed1e7ff',
 '중구': '#0b55a0ff',
 '중랑구': '#1e6eb3ff'}

In [159]:
# 지도영역 나누기
m = folium.Map([37.55784788416739, 127.0088620724302],
                   zoom_start = 10.5,
               tiles = ' cartodbpositron'
#원하는 타일골라쓰기
#Stamenwatercolor
#Stamentoner
#Stamenterrain
#cartodbpositron
#cartodbdark_matter
#       
)

folium.GeoJson(
    seoul_geo,
    name = '시간별거래량',                 #이름
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['name']], # 위에 만든 코드를 넣는곳
        'color': 'black',
        'weight' :1,                       #줄두께
        'dashArray':'8',                # 점선
        'fillOpacity' : 0.6,              #투명도
    }
).add_to(m)

colormap.caption = 'Population color scale'
colormap.add_to(m)

folium.LayerControl().add_to(m)

m

In [42]:
#검색데이터가 많아 연산이 많이 느려지는 것 같다.(중복도 제거해주자)
need=store20[['Latitude','Longitude','store_type']]
need['store_type'].value_counts()

골목상권    84892
발달상권    29533
전통시장    16773
관광특구     1123
Name: store_type, dtype: int64

In [43]:
# need.duplicated().value_counts() #중복이몇개인지확인
need=need.drop_duplicates()#중복값제거

In [44]:
need=need.reset_index()

In [45]:
need['store_type'].value_counts()

골목상권    1009
발달상권     206
전통시장     202
관광특구       6
Name: store_type, dtype: int64

In [51]:
m = folium.Map(
    location = ([37.55784788416739, 127.0088620724302]),
    tiles = 'cartodbpositron',zoom_start = 11
)

In [52]:
for i in range(len(need['Latitude'])):
    c = need['Latitude'][i],need['Longitude'][i]
    
    if need.loc[i, 'store_type'] == '골목상권': # 마크에 원하는 색상을 넣고싶을때
        
        icon_color = 'lightblue'
    elif need.loc[i, 'store_type'] == '발달상권':
        icon_color = 'darkblue'
    elif need.loc[i, 'store_type'] == '전통시장':
        icon_color = 'purple'
    elif need.loc[i, 'store_type'] == '관광특구':
        icon_color = 'yellow'
    folium.features.CircleMarker(
            location =[need['Latitude'][i],need['Longitude'][i]],
            radius = 3,
            popup=c,
            color = icon_color,
            fill=True,
            fill_color= 'white'
        ).add_to(m)
            
m

In [143]:
#지도데이터 불러오기
url = 'https://raw.githubusercontent.com/suanlab/dataset/master'
seoul_geo = f'{url}/seoul_municipalities_geo_simple.json'
seoul_population = f'{url}/seoul_population.csv'

#상권별 점포수 19-20년 차이
StoreCount=store20.groupby(['store_type','구'])['store_count'].size() -\
store19.groupby(['store_type','구'])['store_count'].size()
StoreCount=StoreCount.unstack().T
StoreCount

store_type,골목상권,관광특구,발달상권,전통시장
구,,,,
강남구,-53.0,-5.0,-57.0,10.0
강동구,-144.0,NaN,-48.0,-44.0
강북구,-32.0,NaN,-21.0,-17.0
강서구,-160.0,NaN,-3.0,-21.0
관악구,-108.0,NaN,-44.0,-3.0
광진구,-92.0,NaN,23.0,-46.0
구로구,-87.0,NaN,-55.0,-7.0
금천구,-87.0,NaN,24.0,-5.0
노원구,-16.0,NaN,12.0,-11.0


In [141]:
#인덱싱 재처리
StoreCount= StoreCount.reset_index()
StoreCount.columns

# 지도영역 나누기
m = folium.Map([37.55784788416739, 127.0088620724302],
                   zoom_start = 11,
                   tiles = 'cartodbpositron'
)

folium.Choropleth(
    geo_data=seoul_geo,
    data=StoreCount,
    columns=['구','골목상권'],
    key_on='properties.name',
    fill_color = 'PuRd',
    fill_opacity=0.8,                    #투명도
    line_opacity=0.5,                    # 점선
    legend_name='점포수'             #이름  
    
).add_to(m)

m